In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q

ERROR: Cannot install unsloth because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [2]:
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers datasets -q
!pip install triton

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [215 lines of output]
      running bdist_wheel
      C:\Users\gunes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\cpp_extension.py:495: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
        warnings.warn(msg.format('we could not find ninja.'))
      running build
      running build_py
      creating build\lib.win-amd64-cpython-311\xformers
      copying xformers\attn_bias_utils.py -> build\lib.win-amd64-cpython-311\xformers
      copying xformers\checkpoint.py -> build\lib.win-amd64-cpython-311\xformers
      copying xformers\info.py -> build\lib.win-amd64-cpython-311\xformers
      copying xformers\test.py -> build\lib.win-amd64-cpython-311\xformers
      copying xformers\util

In [3]:
!pip install torch>=2.0.0 transformers>=4.30.0 peft>=0.3.0 datasets>=2.10.0 trl>=0.4.1 bitsandbytes>=0.39.0 accelerate>=0.20.0 scipy scikit-learn

In [4]:
!pip show huggingface_hub

Name: huggingface-hub
Version: 0.25.2
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: C:\Users\gunes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: accelerate, datasets, peft, tokenizers, transformers


In [5]:
# Import the notebook_login function
from huggingface_hub import notebook_login

# Login to Hugging Face
notebook_login()

ModuleNotFoundError: No module named 'huggingface_hub'

In [ ]:
import torch
from datasets import load_dataset
from unsloth import FastLanguageModel, unsloth_save_model
from trl import SFTTrainer
from transformers import TrainingArguments
from huggingface_hub import notebook_login
import os

# Login to Hugging Face
# notebook_login()

# Check if dataset files exist
train_file_path = "train.jsonl"
valid_file_path = "valid.jsonl"

if not os.path.exists(train_file_path):
    print(f"File not found: {train_file_path}. Please ensure the file is in the correct directory.")
if not os.path.exists(valid_file_path):
    print(f"File not found: {valid_file_path}. Please ensure the file is in the correct directory.")

# Load dataset
dataset = load_dataset("json", data_files={"train": train_file_path, "validation": valid_file_path})

# Model setup
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

# PEFT setup
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 42,
    max_seq_length = max_seq_length
)

# Training setup
training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    optim="adamw_torch",
    save_steps=500,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,
    gradient_checkpointing=True,
    max_steps=1000,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    push_to_hub=True,
    hub_model_id="gunzzz24/gemma-2b-4bit-resume",  # Replace with your custom name
)

# Define a formatting function
def formatting_func(example):
    # The data is already formatted, so we just return it as is
    return example['text']

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=256,
    formatting_func=formatting_func
)

# Train"
trainer.train()

# Save model
unsloth_save_model(model, tokenizer, "unsloth_fine_tuned_gemma_2b")